<a href="https://colab.research.google.com/github/NayelySaldivar/DesafioData/blob/main/BasesDeDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> Desafío Data México </h2>
<h4> <p> Segunda Edición - Datos con perspectiva de género </p> </h4> 

##### Importación de librerías:

In [2]:
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Carga y modificación de las bases:

### - Resultados Covid desde la plataforma **DataMexico**:

- Utilizamos la **DataMexico API Explorer** para obtener los datos actualizados:

In [4]:
data=requests.get('https://api.datamexico.org/tesseract/data.jsonrecords?cube=gobmx_covid&drilldowns=Death+Date%2CCovid+Result%2CHealth+Institution+Attended%2CPneumonia+Diagnose%2CSpeaks+Indigenous+Language%2CCOPD+Diagnose%2CAsthma+Diagnose%2CInmunosupresion+Diagnose%2CHypertension+Diagnose%2CCardiovascular+Diagnose%2CObesity+Diagnose%2CChronic+Kidney+Failure+Diagnose%2CDiabetes+Diagnose%2CSmoking+Diagnose%2CContact+with+Another+Covid+Case%2CState+Patient+Attended%2CState+Patient+Origin&measures=Cases&parents=false&sparse=false')
data_api=pd.DataFrame(data.json()["data"])

In [5]:
data_api.head(3)

""


### - Datos históricos de **Mortalidad 2012-2019** del INEGI:

In [6]:
# hist_muertes=pd.read_csv(f'defunciones_generales_{2012}.csv')
hist_muertes=pd.read_csv(f'/content/drive/MyDrive/DesafioData/defunciones_generales_{2012}.csv')
hist_muertes['date']='2012'

for i in range(2013,2020):
    aux=pd.read_csv(f'/content/drive/MyDrive/DesafioData/defunciones_generales_{i}.csv')
    # aux=pd.read_csv(f'defunciones_generales_{i}.csv')
    aux['date']=i
    hist_muertes=pd.concat([hist_muertes,aux])

### - Catálogo de enfermedades según la **CIE - 10**

Concatenamos todas las incidencias:

In [9]:
cat_enfermedad=pd.read_csv(f'/content/drive/MyDrive/DesafioData/decatcausa{2012}.csv', encoding='latin-1')

for i in range(2013,2020):
    aux=pd.read_csv(f'/content/drive/MyDrive/DesafioData/decatcausa{i}.csv',encoding='latin-1')
    cat_enfermedad=pd.concat([cat_enfermedad,aux])
    
cat_enfermedad=cat_enfermedad.reset_index()

Creamos los diccionarios para encontrar las claves únicas:

In [13]:
claves_duplicadas={}

for i in range(len(cat_enfermedad)):
    claves_duplicadas[cat_enfermedad['CVE'][i]]=str(cat_enfermedad['DESCRIP'][i])
    
dict_no_duplicados = {} 
for key, value in claves_duplicadas.items(): 
    if not value in dict_no_duplicados.values(): 
        dict_no_duplicados[key] = value 

Archivamos para usar en el script principal:

In [27]:
claves=pd.DataFrame(list(dict_no_duplicados.keys()),columns=['Clave'])  

desc=[]
for i in claves['Clave'][:-1]:
    desc.append(dict_no_duplicados[str(i)])
    
descr=pd.DataFrame(pd.Series(desc),columns=['Dictamen'])

catalogo_completo=pd.concat([claves,descr],axis=1)

catalogo_completo.to_csv('Catalogo_CIE_10.csv')

#### - Segmentación de padecimientos

In [28]:
EPOC=["J44"+str(i) for i in range(0,10)]

asma=["J450","J451","J458","J459","J46X"]
# cardiovascular=["I0"+str(i)+str(j) for i in range(5,10) for j in range(0,10)]+\ # Cardiopatía reumática
# ["I10X"]+["I1"+str(i)+str(j) for i in range(0,6) for j in range(0,10)]+\ # Hipertensión Arterial
# ["I42X"]+["I42"+str(i)+str(j)  for i in range(0,10) for j in range(0,10)]+\ # Miocardiopatías
# ["I50X"]+["I50"+str(i)+str(j)  for i in range(0,10) for j in range(0,10)]+\ # Insuficiencia cardiaca
# ["I60X"]+["I"+str(i)+str(j)  for i in range(60,70) for j in range(0,10)] +\ # Enfermedad cerebrovascular
# ["I70X","I80X"]+["I"+str(i)+str(j)  for i in range(70,90) for j in range(0,10)] +\ # Enfermedad vascular perifería
# ["Q20X"]+["Q2"+str(i)+str(j)  for i in range(0,9) for j in range(0,10)] # Cardiopatía congénita

cardiovascular=["I0"+str(i)+str(j) for i in range(5,10) for j in range(0,10)]+ \
["I10X"]+["I1"+str(i)+str(j) for i in range(0,6) for j in range(0,10)]+ \
["I42X"]+["I42"+str(i) for i in range(0,10)]+\
["I50X"]+["I50"+str(i)  for i in range(0,10)]+\
["I60X"]+["I"+str(i)+str(j)  for i in range(60,70) for j in range(0,10)] +\
["I70X","I80X"]+["I"+str(i)+str(j)  for i in range(70,90) for j in range(0,10)] +\
["Q20X"]+["Q2"+str(i)+str(j)  for i in range(0,9) for j in range(0,10)] 

hipertension=['G932','I270','I272','K766','O10X','O11X','O13X','O14X','O16X','P292']+ \
["O"+str(i)+str(j) for i in range(10,15) for j in range(0,10)]+\
["O16"+str(i) for i in range(0,10)]

diabetes=["E1"+str(i)+str(j) for i in range(0,5) for j in range(0,10)]+['E232','N251','P702']+["O24"+str(i) for i in range(0,10)]

neumonia=["J1"+str(i)+str(j) for i in range(0,9) for j in range(0,10)]+['J851']+["P23"+str(i) for i in range(0,10)]

insuf_renal=["N1"+str(i)+str(j) for i in range(7,10) for j in range(0,10)]+["I17X","I18X","I19X"]+['P960']

obesidad=["E66"+str(i) for i in range(0,10)]

'''VIH [B20-B24] + Cáncer [C0-C97] + Rechazo de trasplantes [T86] ''' 
inmunosup=["B"+str(i)+str(j) for i in range(20,25) for j in range(0,10)]+['B24X','O987','R75X']+\
["C00"+str(i) for i in range(0,10)]+["C0"+str(i) for i in range(10,100)]+["C"+str(i)+str(j) for i in range(10,97) for j in range(0,10)]+\
["D00"+str(i) for i in range(0,10)]+["D0"+str(i) for i in range(10,100)]+\
["D"+str(i)+str(j) for i in range(80,90) for j in range(0,10)]+\
["T86"+str(i) for i in range(0,10)]

Añadimos el grupo a la base de mortalidad:

In [29]:
hist_muertes['gpo_enfermedad']=np.where(hist_muertes['causa_def'].isin(EPOC),'EPOC',
                                               np.where(hist_muertes['causa_def'].isin(asma),'asma',
                                                 np.where(hist_muertes['causa_def'].isin(cardiovascular),'cardiovascular',
                                                   np.where(hist_muertes['causa_def'].isin(hipertension),'hipertension',
                                                     np.where(hist_muertes['causa_def'].isin(diabetes),'diabetes',
                                                       np.where(hist_muertes['causa_def'].isin(neumonia),'neumonia',
                                                         np.where(hist_muertes['causa_def'].isin(insuf_renal),'insuf_renal',
                                                           np.where(hist_muertes['causa_def'].isin(obesidad),'obesidad',
                                                              np.where(hist_muertes['causa_def'].isin(inmunosup),'inmunosupresores','otros'
                                             )
                                             )
                                             )
                                             )
                                             )
                                             )
                                             )
                                             )
                                             )

In [30]:
# hist_muertes[['gpo_enfermedad','causa_def']].head(3)
hist_muertes.groupby(['gpo_enfermedad','sexo'])['edad'].count()

gpo_enfermedad    sexo
EPOC              1         91039
                  2         82159
                  9             3
asma              1          4945
                  2          5716
                  9             1
cardiovascular    1        254973
                  2        277265
                  9            94
diabetes          1        383722
                  2        401268
                  9             5
hipertension      1          2917
                  2          3672
                  9             1
inmunosupresores  1        272761
                  2        291195
                  9             6
insuf_renal       1         51206
                  2         40068
                  9             7
neumonia          1         99415
                  2         80069
                  9            42
obesidad          1          5703
                  2          4703
otros             1       1842434
                  2       1175977
                  9      

### - **Regiones Nielsen**

**Área I: Pacífico** - Baja California Norte, Baja California Sur, Sonora, Sinaloa, Nayarit (5) - [2, 3, 26, 25, 18]

**Área II: Norte** - Chihuahua, Coahuila, Durango, Nuevo León, San Luis Potosí,Tamaulipas, Zacatecas (7) - [8, 5, 10, 19, 24, 28, 32]

**Área III: Bajío** -  Aguascalientes, Jalisco, Guanajuato, Colima, Michoacan (5) - [1, 14, 11, 6, 16]

**Área IV: Centro** - Hidalgo, Querétaro, Morelos, Edo. México, Tlaxcala, Puebla, Guerrero (7) - [13, 22, 17, 15, 29, 21, 12]

**Área V**: Valle de México - Cdmx (1) - [9]


**Área VI**: Sureste - Chiapas, Campeche, Yucatan, Quintana Roo, Veracruz, Oaxaca, Tabasco (7) - [7, 4, 31, 23, 30, 20, 27]

In [31]:
pacifico = [2, 3, 26, 25, 18]
norte = [8, 5, 10, 19, 24, 28, 32]
bajio = [1, 14, 11, 6, 16]
centro = [13, 22, 17, 15, 29, 21, 12]
vallem = [9]
sureste = [7, 4, 31, 23, 30, 20, 27]

Añadimos la región a la base de mortalidad:

In [32]:
hist_muertes['region']=np.where(hist_muertes['ent_regis'].isin(pacifico),'pacifico',
                                      np.where(hist_muertes['ent_regis'].isin(norte),'norte',
                                               np.where(hist_muertes['ent_regis'].isin(bajio),'bajio',
                                                np.where(hist_muertes['ent_regis'].isin(centro),'centro',
                                                         np.where(hist_muertes['ent_regis'].isin(vallem),'vallem',
                                                             np.where(hist_muertes['ent_regis'].isin(sureste),'sureste','revisar' 
                                                                  )
                                                             )
                                                       )
                                                  )
                                              )
                                        )

In [33]:
hist_muertes.groupby('region')['edad'].count()

region
bajio        911412
centro      1344366
norte        960247
pacifico     485468
sureste     1091112
vallem       581885
Name: edad, dtype: int64

Archivamos los cambios para usar en el script principal:

In [34]:
hist_muertes.to_csv("Bases_INEGI.csv")

### - Resultados de **Censo Nacional de Población 2020** realizado por el INEGI:

In [35]:
# inegi2020=pd.read_csv('inegi2020_con_clavegeo.csv')

## - **Extracto de Mujeres** del histórico de mortalidad:

In [37]:
mortalidad_mujeres=hist_muertes[hist_muertes['sexo']==2]
mortalidad_mujeres.to_csv('Bases_INEGI_Mujeres')